# Reddit

### Import modules

In [1]:
import ciso8601
import requests #Pushshift accesses Reddit via an url so this is needed
import json #JSON manipulation
import datetime
import time
import jsonpickle
import praw
import sys
import os

In [ ]:
## First Approach

### Authentication

In [ ]:
with open('credentials/reddit_credentials') as f:
  credentials2 = json.load(f)

In [ ]:
reddit = praw.Reddit(client_id=credentials2["client_id"], client_secret=credentials2["client_secret"], user_agent=credentials2['user_agent'])
fName = 'data/reddit10000.txt' # We'll store the tweets in a text file.



hot_posts = reddit.subreddit('Covid').top(limit=10000)
posts = 0
comments = 0
with open(fName, 'w') as f:
    for post in hot_posts:
        post.comments.replace_more(limit=None)
        #print("Title",post.title)
        f.write(jsonpickle.encode(post.title, unpicklable=False) +'\n')
        #print("Self Test",post.selftext)
        f.write(jsonpickle.encode(post.selftext, unpicklable=False) +'\n')
        posts = posts +1
        for top_level_comment in post.comments:
            #print("C:",top_level_comment.body)
            f.write(jsonpickle.encode(top_level_comment.body, unpicklable=False) +'\n')
            comments = comments + 1


print ("Downloaded {0} posts and {1} comments, Saved to {2}".format(posts,comments, fName))


In [ ]:
reddit = praw.Reddit(client_id=credentials2["client_id"], client_secret=credentials2["client_secret"], user_agent=credentials2['user_agent'])
fName = 'data/reddit1.txt' # We'll store the tweets in a text file.



hot_posts = reddit.subreddit('Covid').top(limit=10000)
posts = 0
comments = 0
with open(fName, 'w') as f:
    for post in hot_posts:
        post.comments.replace_more(limit=None)
        #print("Title",post.title)
        f.write(jsonpickle.encode(post.title, unpicklable=False) +'\n')
        #print("Self Test",post.selftext)
        f.write(jsonpickle.encode(post.selftext, unpicklable=False) +'\n')
        posts = posts +1
        for top_level_comment in post.comments:
            #print("C:",top_level_comment.body)
            f.write(jsonpickle.encode(top_level_comment.body, unpicklable=False) +'\n')
            comments = comments + 1


print ("Downloaded {0} posts and {1} comments, Saved to {2}".format(posts,comments, fName))



## Second Approach

### Using Pushshift Module to extract Submissions Data from Reddit via Python

### Pushshift get URL

In [62]:
#Adapted from this https://gist.github.com/dylankilkenny/3dbf6123527260165f8c5c3bc3ee331b
#This function builds an Pushshift URL, accesses the webpage and stores JSON data in a nested list
def getPushshiftData(query, after, before, sub):
    #Build URL
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    #Print URL to show user
    print(url)
    #Request URL
    r = requests.get(url)
    #Load JSON data from webpage into data variable
    data = json.loads(r.text)
    #return the data element which contains all the submissions data
    return data['data']

### Extract key information from Submissions

We want key data for further analysis including:
* Submission Title
* Author
* Submission post ID
* Upload Time
* Text


In [63]:
#This function will be used to extract the key data points from each JSON result
def collectSubData(subm,id):
    #subData was created at the start to hold all the data which is then added to our global subStats dictionary.
    subData = {} #list to store data points
    subData['id'] = id
    subData['internal_id'] = subm['id']
    subData['publication_date'] = int(subm['created_utc'])#1520561700.0
    subData['source'] = "Reddit"
    subData['author'] = subm['author']
    subData['title'] = subm['title']
    subData['text'] = subm['selftext']
    #Put all data points into a tuple and append to subData
    #Create a dictionary entry of current submission data and store all data related to it
    subStats[id] = subData

### Search Settings

In [64]:
#Create your timestamps and queries for your search URL
#https://www.unixtimestamp.com/index.php > Use this to create your timestamps
after = "1577836800" #int(time.mktime(ciso8601.parse_datetime(date_after).timetuple())) #Submissions after this timestamp
before = "1607040000" #int(time.mktime(ciso8601.parse_datetime(date_after).timetuple())) #Submissions after this timestamp
query = "COVID" #Keyword(s) to look for in submissions
sub = "COVID" #Which Subreddit to search in

#subCount tracks the no. of total submissions we collect
subCount = 0
#subStats is the dictionary where we will store our data.
subStats = {}

In [65]:
# We need to run this function outside the loop first to get the updated after variable
data = getPushshiftData(query, after, before, sub)
# Will run until all posts have been gathered i.e. When the length of data variable = 0
# from the 'after' date up until before date
id = 0
while len(data) > 0: #The length of data is the number submissions (data[0], data[1] etc), once it hits zero (after and before vars are the same) end
    for submission in data:
        collectSubData(submission, id)
        id+=1
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    #update after variable to last created date of submission
    after = data[-1]['created_utc']
    #data has changed due to the new after variable provided by above code
    data = getPushshiftData(query, after, before, sub)

print(len(data))

https://api.pushshift.io/reddit/search/submission/?title=COVID&size=1000&after=1577836800&before=1607040000&subreddit=COVID
100
2020-03-26 20:59:00
https://api.pushshift.io/reddit/search/submission/?title=COVID&size=1000&after=1585274340&before=1607040000&subreddit=COVID
100
2020-04-01 21:53:37
https://api.pushshift.io/reddit/search/submission/?title=COVID&size=1000&after=1585796017&before=1607040000&subreddit=COVID
100
2020-04-08 04:55:42
https://api.pushshift.io/reddit/search/submission/?title=COVID&size=1000&after=1586339742&before=1607040000&subreddit=COVID
100
2020-04-12 07:54:14
https://api.pushshift.io/reddit/search/submission/?title=COVID&size=1000&after=1586696054&before=1607040000&subreddit=COVID
100
2020-04-17 12:47:55
https://api.pushshift.io/reddit/search/submission/?title=COVID&size=1000&after=1587145675&before=1607040000&subreddit=COVID
100
2020-04-23 15:58:54
https://api.pushshift.io/reddit/search/submission/?title=COVID&size=1000&after=1587675534&before=1607040000&subr

### Check your Submission Extraction was successful

In [66]:
num_submissions = len(subStats)
print(str(num_submissions) + " submissions have added to list")
print("1st entry is:")
print(list(subStats.values())[0]["title"])
print("Last entry is:")
print(list(subStats.values())[-1]["title"])

3398 submissions have added to list
1st entry is:
Coronavirus disease named Covid-19
Last entry is:
Covid Help - Testing - How To Properly Quarantine?


### Save data to JSON file

In [67]:
def updateSubs_file():
    ts = str(round(time.time() * 1000))
    file = "data/reddit_" + str(num_submissions) +"_" + ts + ".JSON"
    with open(file, 'w') as file:
        json.dump(subStats,file)
    print("JSON created")

updateSubs_file()

JSON created


## Comments

(Not yet implemented)

https://reddit-api.readthedocs.io/en/latest/index.html#comments-search


In [ ]:
import requests
from collections import defaultdict
from datetime import datetime
from datetime import timedelta
import time
import json

In [ ]:
subreddits = ['Coronavirus']
ignored_users = ['[deleted]', 'automoderator']
lookback_days = 180
min_comments_per_sub = 1
file_name = "/data/reddint1.txt"
# either author, comments or leave it empty
sort_by = ""

url = "https://api.pushshift.io/reddit/comment/search?&limit=1000&sort=desc&subreddit={}&before="

startTime = datetime.utcnow()
startEpoch = int(startTime.timestamp())
endTime = startTime - timedelta(days=lookback_days)
endEpoch = int(endTime.timestamp())
totalSeconds = startEpoch - endEpoch


def countCommenters(subreddit):
	count = 0
	commenters = defaultdict(int)
	previousEpoch = startEpoch
	print(f"Counting commenters in: {subreddit}")
	breakOut = False
	while True:
		newUrl = url.format(subreddit)+str(previousEpoch)
		try:
			response = requests.get(newUrl, headers={'User-Agent': "Overlap counter by /u/Watchful1"})
		except requests.exceptions.ReadTimeout:
			print(f"Pushshift timeout, this usually means pushshift is down. Waiting 5 seconds and trying again: {newUrl}")
			time.sleep(5)
			continue
		try:
			objects = response.json()['data']
		except json.decoder.JSONDecodeError:
			print(f"Decoding error, this usually means pushshift is down. Waiting 5 seconds and trying again: {newUrl}")
			time.sleep(5)
			continue

		time.sleep(1)  # pushshift is ratelimited. If we go too fast we'll get errors

		if len(objects) == 0:
			break
		for object in objects:
			previousEpoch = object['created_utc'] - 1
			if object['author'] not in ignored_users:
				commenters[object['author']] += 1
			count += 1
			if count % 1000 == 0:
				print("r/{0} comments: {1}, {2}, {3:.2f}%".format(
					subreddit,
					count,
					datetime.fromtimestamp(previousEpoch).strftime("%Y-%m-%d"),
					((startEpoch - previousEpoch) / totalSeconds) * 100))
			if previousEpoch < endEpoch:
				breakOut = True
				break
		if breakOut:
			break
	print(f"Comments: {count}, commenters: {len(commenters)}")
	return commenters


totalCommenters = defaultdict(int)
overlapCommenters = defaultdict(int)
for subreddit in subreddits:
	commenters = countCommenters(subreddit)

	if not len(overlapCommenters):
		print("Building first")
		for commenter in commenters:
			if commenters[commenter] >= min_comments_per_sub:
				overlapCommenters[commenter] += commenters[commenter]
				totalCommenters[commenter] += commenters[commenter]
		print(f"Done, size: {len(overlapCommenters)}")
	else:
		print(f"Building, current size: {len(overlapCommenters)}")
		newOverlap = defaultdict(int)
		for commenter in commenters:
			if commenters[commenter] >= min_comments_per_sub:
				totalCommenters[commenter] += commenters[commenter]
				if commenter in overlapCommenters:
					newOverlap[commenter] += commenters[commenter]
		overlapCommenters = newOverlap
		print(f"Done, new size: {len(overlapCommenters)}, total commenters: {len(totalCommenters)}")


print(f"{len(overlapCommenters)} of {len(totalCommenters)} total commenters commented in all subreddits, that's {round((len(overlapCommenters) / len(totalCommenters)) * 100, 2)} percent")

with open(file_name, 'w') as txt:
	if sort_by == 'author':
		print(f"Printing to {file_name} sorted by author")
		for user in sorted(overlapCommenters.keys(), key=str.lower):
			txt.write(f"{user}: {overlapCommenters[user]}\n")
	elif sort_by == 'comments':
		print(f"Printing to {file_name} sorted by number of comments")
		for user, comments in sorted(overlapCommenters.items(), key=lambda item: item[1], reverse=True):
			txt.write(f"{user}: {comments}\n")
	else:
		print(f"Printing to {file_name} unsorted")
		for user in overlapCommenters:
			txt.write(f"{user}: {overlapCommenters[user]}\n")